This Jupyter Notebook serves as a proof of concept of a multi-agent solution for MedTech regulations. The intent of the system is to provide clear answers to questions on WHO and FDA documentations on medical devices, using a 3-to-4 agents system composed like this:

- an LLM orchestrator that receives the question and coordinates agents
- a RAG agent capable of retrieving documents related to the question
- an LLM response agent to put together the answer based on the documents retrieved and the question
- a possible fourth agent to be decided (summary agent, source-verifier, compare agent, prompt agent to improve prompts, etc.)

In [1]:
"""

pseudo code for multi agent system

class agent

instantiate orchestrator and other agents

define orchestrator prompt and response false

take input

while not response:

    orchestrator call

    designed agent call

return response

"""

'\n\npseudo code for multi agent system\n\nclass agent\n\ninstantiate orchestrator and other agents\n\ndefine orchestrator prompt and response false\n\ntake input\n\nwhile not response:\n\n    orchestrator call\n\n    designed agent call\n\nreturn response\n\n'

In [53]:
# Import libraries

import gradio as gr
import chromadb
import uuid
import google.generativeai as genai
import os
from dotenv import load_dotenv
import pymupdf4llm
import json
from chromadb.utils import embedding_functions

In [9]:
# Load environmental variables and AI models

load_dotenv() #loads the API key put in a .env file
try:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
except Exception as e:
    print(f"Error configuring Google AI. Please ensure your API key is correct. Error: {e}")

# Configure paths to data
current_dir = os.getcwd() 
FILE_PATH_FDA_DESIGN = os.path.join(current_dir, '..', 'documents', 'FDA_Design_Control_Guidance.pdf')
FILE_PATH_WHO = os.path.join(current_dir, '..', 'documents', 'WHO_Medical_Device_Regulations.pdf')
FILE_PATH_FDA_POLICY = os.path.join(current_dir, '..', 'documents', 'FDA_Policy_Device_Software_Functions.pdf')
COLLECTION_NAME = "multi_agent_rag"

In [29]:
# Split logic (very simple, splitting paragraphs)

def split_into_chunks(dict_list):
    text_chunks = []
    metadatas = []
    
    for document in dict_list:
        try:
            paragraphs = list(document.values())[0].split("\n\n")
    
            for paragraph in paragraphs: # Delete short paragraphs
                if len(paragraph) > 10:
                    text_chunks.append(paragraph)
                    metadatas.append({"source": list(document.keys())[0]})
        except Exception as e:
            print(f"An error occurred while processing {list(document.keys())[0]}: {e}")
            
    return text_chunks, metadatas

# Use pymupdf4llm to convert pdf into text, adequately formatted. Using dicts to keep track of sources and add them to metadata while chunking
fda_design_dict = {'FDA_Design_Control_Guidance.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_DESIGN)}
who_dict = {'WHO_Medical_Device_Regulations.pdf': pymupdf4llm.to_markdown(FILE_PATH_WHO)}
fda_policy_dict = {'FDA_Policy_Device_Software_Functions.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_POLICY)}

text_chunks, metadatas = split_into_chunks([fda_design_dict, who_dict, fda_policy_dict])
print(len(text_chunks), len(metadatas))

1657 1657


In [114]:
# Second chunking logic: texting up the documents. I'll convert the pdf in txt and see if the RAG handles them better.

# Configure paths to data
current_dir = os.getcwd() 
FILE_PATH_FDA_DESIGN = os.path.join(current_dir, '..', 'documents', 'FDA_Design_Control_Guidance.txt')
FILE_PATH_WHO = os.path.join(current_dir, '..', 'documents', 'WHO_Medical_Device_Regulations.txt')
FILE_PATH_FDA_POLICY = os.path.join(current_dir, '..', 'documents', 'FDA_Policy_Device_Software_Functions.txt')
COLLECTION_NAME = "multi_agent_rag"

def process_txt_file(file_path, source_name):
    """
    Reads a .txt file and splits it into chunks based on paragraphs
    (separated by double newlines).

    Args:
        file_path (str): The path to the .txt file.
        source_name (str): The name of the source document for metadata.

    Returns:
        tuple: A tuple containing two lists: (text_chunks, metadatas).
    """
    all_text_chunks = []
    all_metadatas = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            full_text = f.read()

        # Chunking strategy: split by double newline (a common paragraph separator)
        paragraphs = full_text.split('\n\n')

        for para in paragraphs:
            stripped_para = para.strip()
            if len(stripped_para) > 25:  # Filter out very short or empty lines
                all_text_chunks.append(stripped_para)
                # Metadata is simpler for a txt file, just the source
                all_metadatas.append({'source': source_name})

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

    return all_text_chunks, all_metadatas

chunks_fdad, metas_fdad = process_txt_file(FILE_PATH_FDA_DESIGN, 'FDA_Design_Control_Guidance')
print(f"Processed FDA_Design: {len(chunks_fdad)} chunks.")

chunks_who, metas_who = process_txt_file(FILE_PATH_WHO, 'WHO_Medical_Device_Regulations')
print(f"Processed WHO: {len(chunks_who)} chunks.")

chunks_fdap, metas_fdap = process_txt_file(FILE_PATH_FDA_POLICY, 'FDA_Policy_Device_Software_Functions')
print(f"Processed FDA Policy: {len(chunks_fdap)} chunks.")

text_chunks = chunks_fdad + chunks_who + chunks_fdapS
metadatas = metas_fdad + metas_who + metas_fdap

Processed FDA_Design: 67 chunks.
Processed WHO: 120 chunks.
Processed FDA Policy: 100 chunks.


In [115]:
# Using Langchain recursive tex splitter NEED TO ADD METADATAS THOUGH

from langchain_text_splitters import RecursiveCharacterTextSplitter
file_paths = [FILE_PATH_FDA_DESIGN, FILE_PATH_WHO, FILE_PATH_FDA_POLICY]

all_text = ""
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as f:
        all_text += f.read() + "\n\n"

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.create_documents([all_text])
print(texts[0])
print(texts[1])
len(texts)

page_content='Center for Devices and Radiological Health
DESIGN CONTROL GUIDANCE
FOR
MEDICAL DEVICE MANUFACTURERS
This Guidance relates to
FDA 21 CFR 820.30 and Sub-clause 4.4 of ISO 9001'
page_content='FOREWORD
To ensure that good quality assurance practices are used for the design of medical devices
and that they are consistent with quality system requirements worldwide, the Food and'


2093

In [83]:
# Define agent classes

class OrchestratorAgent:

    def __init__(self, agents):
        self.agents = agents
        self.memory = []
        self.memory_limit = 15

        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def run(self):
        print("Hi! I am your MedTech regulatory companion. How can I help you?")
        user_input = input("You: ")

        while True:
            self.memory = self.memory[-self.memory_limit:]
            if user_input.lower() in ["exit", "bye", "close"]:
                print("I hope I could be of use to you, have a great day!")
                break
                
            orch_response = self.orchestrate(user_input)
            if orch_response["agent_to_call"] == "No action needed":
                print("Is there anything else I can help you with?")
                user_input = input("You: ")
            for agent in self.agents:
                if agent.name == orch_response["agent_to_call"]:
                    print(f"Found agent I was looking for: {agent.name}\n")
                    response = agent.act(orch_response["output"], orch_response["relevant_info"], self.memory)
                    self.memory.append(f"Agent {agent.name} responded {response}")
                    #controllo da togliere
                    if agent.name == "response_agent":
                        return
        
        return response

    def orchestrate(self, user_input):
        self.memory.append(f"User: {user_input}")
        self.memory = self.memory[-self.memory_limit:]
        context = "\n".join(self.memory)

        response_format = {"agent_to_call":"", "output": "", "relevant_info":""}
        response = self.model.generate_content(self.get_prompt(context, response_format))
        self.memory.append(f"Orchestrator: {response.text}")
        response_cleaned = clean_response(response)
        #print(f"The next agent to call is {response_cleaned['agent_to_call']}")
        return response_cleaned

    def get_prompt(self, context, response_format):
        prompt = f"""
        Act as an orchestrator agent for an intelligent RAG system for MedTech companies. Your task is to coordinate agents in order to extract relevant documents from a RAG system and package a coherent and precise answer to the query received.
        The task is to call a first time the ragagent, and then use the ragagent documents to call the response_agent on them and craft a response. Make sure to call the response_agent if the history contains a previous call to the ragagent.
        Your AI agents and their descriptions are {", ".join([f"- {agent.name}: {agent.description}" for agent in self.agents])}

        Use the context, which includes the current user input and the memory of previous inputs and outputs, to plan next steps.
        Context : {context}

        Guidelines:
        At every step, you need to choose only one of the agents provide instruction to only that agent. If the request needs multiple agent to be solved, do that in a loop.
        Read the context, take your time to understand the task, and check if you have executed it correctly.
        If there are no actions needed, default the "agent_to_call" parameter to "No action needed" in the response.
        Return only the agent name in the "agent_to_call" parameter.
        You will return instructions in a valid JSON in the form of {response_format}. All output should be of string type, the "output" is for the query and the "relevant_info" is to attach documents from the RAG for the response agent.
        """
        return prompt
        
    def call_agent(self, history, information):
        prompt = self.prompt + "\n\n" + " ".join(json.dumps(history)) + "\n\n" + information
        response = self.model.generate_content(prompt)
        return response

class RAGAgent:

    def __init__(self):
        self.name = "ragagent"
        self.description = """I am a RAG agent that can search for relevant documents in a vector database in order to answer a query.
                            I expect a user query as input and will return relevant chunks and a variable containing sources info, relevance scores and chunks.
                            """
        self.memory = []
        self.memory_limit = 15
        self.sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
        self.collection_name = COLLECTION_NAME

    def act(self, query, relevant_info, memory):
        if "RAG initialized" not in history:
            self.collection = self.initialize_db()
            self.memory.append("RAG initialized")
        self.memory.append(memory)
        self.memory = self.memory[-self.memory_limit:]
        documents = self.query_db(query)
        return documents

    def initialize_db(self):
        print("Initializing RAG system... This may take a minute.")
        self.client = chromadb.Client()

        if self.collection_name in [c.name for c in self.client.list_collections()]:
            self.client.delete_collection(name = self.collection_name)
            print(f"Deleted existing collection: {self.collection_name}"
                 )
    
        collection = self.client.get_or_create_collection(
        name = self.collection_name,
        embedding_function = self.sentence_transformer_ef
        )
        self.load_documents(collection, text_chunks, metadatas)

        return collection

    def load_documents(self, collection, document_chunks, metadatas):
        collection.add(
        ids = [str(uuid.uuid4()) for _ in text_chunks],
        documents = text_chunks,
        metadatas = metadatas)
    
    def query_db(self, question):
        results = self.collection.query(query_texts=[question], include = ["documents", "metadatas", "distances"], n_results=5)

        sources_markdown = "### Sources Used for Analysis\n\n"
        retrieved_documents = results['documents'][0]
        retrieved_metadatas = results['metadatas'][0]
        retrieved_distances = results['distances'][0]
    
        for i, (doc, meta, dist) in enumerate(zip(retrieved_documents, retrieved_metadatas, retrieved_distances)):
            # Convert distance to a more intuitive similarity score (1 - distance)
            relevance_score = 1 - dist
            source_info = f"**Source {i+1}:** {meta.get('source', 'N/A')}, Page {meta.get('page', 'N/A')}\n"
            relevance_info = f"**Relevance Score:** {relevance_score:.2f}\n\n"
            content_info = f"```\n{doc}\n```\n\n---\n\n"
            sources_markdown += source_info + relevance_info + content_info
        print(f"Sources, documents and relevance score: {sources_markdown}")
            
        return sources_markdown

class ResponseAgent:
    def __init__(self):
        self.name = "response_agent"
        self.description = """I am a response agent that expects as input a user query and relevant documents and info from a RAG search.
                            My task is to craft a precise response for the user based on the provided documents. I will return a response text.
                            """

        self.memory = []
        self.memory_limit = 15
        self.model = genai.GenerativeModel('gemini-1.5-flash')
    
    def act(self, query, relevant_info, memory):
        prompt = self.get_prompt(query, relevant_info, memory)
        response = self.model.generate_content(prompt)
        print(response.text)
        return response.text

    def get_prompt(self, query, relevant_info, memory):
        prompt = f"""
        Act as a Senior Consultant for medical devices. You receive a query from your client, and answer to it based on the relevant information you receive from the RAG system as document chunks.
        Be precise, confident and do not make things up. If the context is not enough to provide a clear answer, state it.
        Cite the documents and sources you receive as part of your input and provide strategic recommendation. The structure of your answer will be:
        - Salutation
        - Precise response to the query based on the documents received from the RAG
        - Strategic recommendation to the customer.

        As additional resources:
        User input: {query}
        Relevant documents with sources and relevance scores: {relevant_info}
        Memory of previous inputs and info: {memory}
        """
        return prompt

In [ ]:
# why not add an agent to do chunking? add pydantic to force the format of the answer
# it would make more sense to generalize the response_agent class since it is an llm calling, let's see if we have time

class ChunkingAgent():
    


In [86]:
# Instantiate agent objects

def clean_response(response):
    response_text = response.text
    print(response.text)
    start_index = response_text.find("{")
    end_index = response_text.rfind("}")
    if start_index != -1 and end_index != -1:
        return json.loads(response_text[start_index : end_index + 1])
    else:
        return response_text

orchestrator = OrchestratorAgent()
ragagent = RAGAgent()
response_agent = ResponseAgent()

final_answer = None
history = []
query = "What documentation is needed for a mobile app that monitors heart rate?"
count_rag = 0
count_final = 0

while not final_answer:
    response_json = clean_response(orchestrator.call_agent(history, query))
    history.append(response_text)
    print(response_text)
    if json_response["agent_name"] == "ragagent":
        count_rag += 1
        #retrieved_documents, sources_markdown = ragagent.act(history, query)
        history.extend(ragagent.act(history, query))
    elif json_response["agent_name"] == "response_agent":
        count_final += 1
        final_answer = response_agent.craft_response(query, history)
    else:
        print("there is an error, the agent called does not exist")
    if count_rag == 3 or count_final == 3:
        final_answer = "Finito"

print(final_answer.text )

```json
{
  "agent_name": "ragagent",
  "query": "Documentation requirements for a mobile heart rate monitoring application",
  "notes": "Focus on regulatory compliance (e.g., HIPAA, GDPR, FDA),  security, privacy, and user interface/usability aspects of the documentation.  Next step should be to the response agent."
}
```

8 319
```json
{
  "agent_name": "ragagent",
  "query": "documentation requirements for a mobile heart rate monitoring application",
  "notes": ""
}
```

Initializing RAG system... This may take a minute.
Deleted existing collection: multi_agent_rag
```json
{
  "agent_name": "ragagent",
  "query": "Documentation requirements for a mobile heart rate monitoring application",
  "notes": "Focus on regulatory, privacy, and user interface documentation.  Next step should be to the response_agent to synthesize a concise answer."
}
```

8 278
```json
{
  "agent_name": "ragagent",
  "query": "documentation requirements for a mobile heart rate monitoring application",
  "notes

AttributeError: 'str' object has no attribute 'text'